In [90]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps

In [91]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [92]:
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [143]:
brandGroupsIds = "Lipigas"
busquedaDatos = "FIND {" + brandGroupsIds + "} RETURNING Brand_Account__c(Id,Atributo_1_3__c,Valor_1_3__c,Nombre_Elemento_1__c,Tipo_Elemento_1__c,Brand_Groups__r.Empresa__c  )"
derty = dict(sf.search(busquedaDatos))

In [144]:
derty

{'searchRecords': [OrderedDict([('attributes',
                OrderedDict([('type', 'Brand_Account__c'),
                             ('url',
                              '/services/data/v42.0/sobjects/Brand_Account__c/a1PRN000000IA0w2AG')])),
               ('Id', 'a1PRN000000IA0w2AG'),
               ('Atributo_1_3__c', None),
               ('Valor_1_3__c', None),
               ('Nombre_Elemento_1__c', None),
               ('Tipo_Elemento_1__c', None),
               ('Brand_Groups__r', None)]),
  OrderedDict([('attributes',
                OrderedDict([('type', 'Brand_Account__c'),
                             ('url',
                              '/services/data/v42.0/sobjects/Brand_Account__c/a1PRN000000IA0v2AG')])),
               ('Id', 'a1PRN000000IA0v2AG'),
               ('Atributo_1_3__c', None),
               ('Valor_1_3__c', None),
               ('Nombre_Elemento_1__c', None),
               ('Tipo_Elemento_1__c', None),
               ('Brand_Groups__r', None)]),
 

In [146]:
brandGroupsIds = "a41RN0000000fNRYAY"



soql_result = sf.query(
    format_soql(
        "SELECT Id,Nombre_Elemento_1__c,Tipo_Elemento_1__c,Atributo_1_3__c,Valor_1_3__c   from Brand_Account__c WHERE Brand_Groups__c = {brandGroupId} and Tipo_de_Front__c = 'mobile' and Tipo_Elemento_1__c = 'C'",
        brandGroupId=brandGroupsIds,
    )
)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
# df = df.drop(columns="Id")
list_of_records = df.to_dict("index")






for listado in list_of_records:
    print(
        
         list_of_records[listado]["Id"]
        + " - "
        + list_of_records[listado]["Valor_1_3__c"]
        + " - "
        + list_of_records[listado]["Nombre_Elemento_1__c"]
    )


a1PRN000000I0hV2AS - 0VSRN00000000uT4AQ - Legal Familia
a1PRN000000I0hb2AC - 0VS8c000000obNVGAY - Asesoría Psicológica en Salud Sexual
a1PRN000000I0hc2AC - 0VS8c000000obNcGAI - Orientación en Salud


In [81]:
list_of_records

{0: {'Id': 'a1PRN000000I0hV2AS',
  'Nombre_Elemento_1__c': 'Legal Familia',
  'Tipo_Elemento_1__c': 'C',
  'Atributo_1_3__c': 'Id',
  'Valor_1_3__c': '0VSRN00000000uT4AQ'},
 1: {'Id': 'a1PRN000000I0hb2AC',
  'Nombre_Elemento_1__c': 'Asesoría Psicológica en Salud Sexual',
  'Tipo_Elemento_1__c': 'C',
  'Atributo_1_3__c': 'Id',
  'Valor_1_3__c': '0VS8c000000obNVGAY'},
 2: {'Id': 'a1PRN000000I0hc2AC',
  'Nombre_Elemento_1__c': 'Orientación en Salud',
  'Tipo_Elemento_1__c': 'C',
  'Atributo_1_3__c': 'Id',
  'Valor_1_3__c': '0VS8c000000obNcGAI'}}

In [ ]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""

In [ ]:
archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)
archivoBCI["Programas_por_Empresas__c"] = "a1W8c000008TCWxEAO"
archivoBCI.head()

In [79]:
desc = sf.Brand_Account__c.describe()
fields_name = [field['name'] for field in desc['fields']]
soql = "SELECT {} FROM Brand_Account__c WHERE Brand_Groups__c = 'a41RN0000000fNRYAY' ".format(','.join(fields_name))
results = sf.query_all(soql)


""" fields = [field.get('Brand_Account__c') for field in getattr(sf, Brand_Account__c).describe().get('fields')]

# Concat fields ready for inclusion in query string
fields = ',\n'.join(fields)

# Convert into SOQL query
soql = f"SELECT \n{fields} \nFROM {TABLE}" """
results

{'records': [OrderedDict([('attributes',
                OrderedDict([('type', 'Brand_Account__c'),
                             ('url',
                              '/services/data/v42.0/sobjects/Brand_Account__c/a1PRN000000I0ha2AC')])),
               ('Id', 'a1PRN000000I0ha2AC'),
               ('OwnerId', '0058c00000DSxZrAAL'),
               ('IsDeleted', False),
               ('Name', 'Home'),
               ('RecordTypeId', '012RN000000YHWIYA4'),
               ('CreatedDate', '2023-12-29T17:51:13.000+0000'),
               ('CreatedById', '0058c00000DSxZrAAL'),
               ('LastModifiedDate', '2023-12-29T18:49:50.000+0000'),
               ('LastModifiedById', '0058c00000DSxZrAAL'),
               ('SystemModstamp', '2024-01-03T12:54:28.000+0000'),
               ('LastActivityDate', None),
               ('LastViewedDate', '2024-01-03T12:10:46.000+0000'),
               ('LastReferencedDate', '2024-01-03T12:10:46.000+0000'),
               ('Account__c', None),
         